# Script para aplicar a lematização nas bases de palavras classificadas, a fim de gerar uma nova base  com todas as variações de uma palavra, não só sua forma canonica.


In [ ]:
!pip install spacy pyinflect --quiet
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.5/703.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 100.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Importar o drive para importar o arquivo das pastas
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import time
import spacy
import pyinflect

# Carrega o modelo spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:

# Função para gerar formas morfossintáticas (incluindo a palavra original)
def get_flexed_forms(word):
    doc = nlp(word)
    token = doc[0]
    forms = set()

    # Adiciona a palavra original
    forms.add(word)

    # Lema
    lemma = token.lemma_
    forms.add(lemma)

    # Plural (mesmo que POS não seja NOUN)
    plural = token._.inflect('NNS')
    if plural and plural != word:
        forms.add(plural)

    # Flexões verbais: 3ª pessoa, gerúndio, passado, particípio
    for tense in ['VBZ', 'VBG', 'VBD', 'VBN']:
        inflected = token._.inflect(tense)
        if inflected and inflected != word:
            forms.add(inflected)

    return sorted(forms)

# Caminho da planilha de entrada
FILE_KW = "/content/drive/My Drive/TCC 2025/Base_de_dados/com_ator_saida_palavras_agrupadas_questoes_especificas_sprint_final.csv"

# Lê a planilha
df = pd.read_csv(FILE_KW)

# Verifica se a coluna 'palavra' existe
if 'palavra' not in df.columns:
    raise ValueError("A coluna 'palavra' não foi encontrada no CSV.")

# Remove colunas desnecessárias
colunas_para_remover = ['frequência', 'tipo', 'Validação']
df = df.drop(columns=[col for col in colunas_para_remover if col in df.columns])

# Converte todas as colunas (exceto 'palavra') para int
for col in df.columns:
    if col != 'palavra':
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# Lista para armazenar novas linhas
linhas_expandidas = []

# Inicia a contagem de tempo
inicio = time.time()

# Gera novas linhas com as formas flexionadas
for _, row in df.iterrows():
    palavra = row['palavra']
    flexoes = get_flexed_forms(palavra)
    for forma in flexoes:
        nova_linha = row.copy()
        nova_linha['palavra'] = forma
        linhas_expandidas.append(nova_linha)

# Finaliza a contagem de tempo
fim = time.time()
duracao = fim - inicio

# Cria DataFrame final com as novas formas
df_expandidas = pd.DataFrame(linhas_expandidas)

# Remove linhas duplicadas (considerando todas as colunas)
df_expandidas = df_expandidas.drop_duplicates()

# Caminho de saída do CSV
output_path = "/content/drive/My Drive/TCC 2025/Base_de_dados/saida_palavras_flexionadas.csv"
df_expandidas.to_csv(output_path, index=False)

# Exibe resultado
print(f"Arquivo salvo em: {output_path}")
print(f"Tempo total de execução: {duracao:.2f} segundos")


Arquivo salvo em: /content/drive/My Drive/TCC 2025/Base_de_dados/saida_palavras_flexionadas.csv
Tempo total de execução: 2.28 segundos
